# CrewAI Proxy example connecting to Sap Generative AI Hub via LiteLLM

## [How CrewAI works](https://docs.crewai.com/en/introduction)
## CrewAI also supports the [LiteLLM Proxy](https://docs.litellm.ai/docs/simple_proxy/) for access via OpenAI API calls.

## Installation
ٔٔFollow details in [LiteLLM Proxy set up for SAP models](../proxy_set_up.ipynb)

In [1]:
%pip install crewai

INFO: pip is looking at multiple versions of opentelemetry-exporter-otlp-proto-grpc to determine which version is compatible with other requirements. This could take a while.
  Attempting uninstall: opentelemetry-exporter-otlp-proto-grpc
    Found existing installation: opentelemetry-exporter-otlp-proto-grpc 1.38.0
    Uninstalling opentelemetry-exporter-otlp-proto-grpc-1.38.0:
      Successfully uninstalled opentelemetry-exporter-otlp-proto-grpc-1.38.0

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
LITELLM_PROXY_API_KEY=sk-1234
PROXY_BASE_URL=http://localhost:4000
```

## Run the CrewAI with LiteLLM and SAP LLMs

In [2]:
import os

import litellm
from crewai import LLM, Agent, Crew, Task
from crewai.tools import tool
from dotenv import load_dotenv

Load your credentials as environment variables.

In [3]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("PROXY_BASE_URL")
api_key = os.getenv("LITELLM_PROXY_API_KEY")

Set up the model with your proxy params

In [4]:
proxy_llm = LLM(
    model="sap/gpt-4o", api_base=api_base, base_url=api_base, api_key=api_key
)

Define the agent tools.

In [5]:
@tool("get_weather")
def get_weather(city: str) -> str:
    """Moke function"""
    city_normalized = city.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

User can select a city.

In [6]:
city = "Tbilisi"

Define the Agent and the SAP LLM to be used by CrewAI.

In [7]:
agent = Agent(
    role="Weather presenter",
    goal=f"Prepare a couple sentences TV speach about weather in the {city}, "
         f"using information from run the get_weather tool",
    backstory="You are the weather presenter on TV",
    llm=proxy_llm,
    tools=[get_weather],
    allow_delegation=False,
)

Define tasks for the agents.

In [8]:
agent_task = Task(
    description=(
        f"Write a couple sentences for TV weather report in {city}, that will be include small jok"
    ),
    expected_output=(
        "Good quality text of two sentences about weather and with small jok"
    ),
    agent=agent,
)

Create the crew with the Agent and task.

In [9]:
crew = Crew(
    agents=[agent],
    tasks=[agent_task],
    verbose=True,
)

Run the crew

In [10]:
result = crew.kickoff()
print("\n📘 Result:\n", result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a34de1e7-fd36-4e2a-a932-293c7a3e221b                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Weather presenter                                                                                       │
│                                                                                                                 │
│  Task: Write a couple sentences for TV weather report in Tbilisi, that will be include small jok                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.AuthenticationError: AuthenticationError: Litellm_proxyException - The api_key client option    │
│  must be set either by passing api_key to the client or by setting the LITELLM_PROXY_API_KEY environment        │
│  variable                                                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/vasilisa/PycharmProjects/litellm-agentic-examples/.venv/lib/python3.12/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')


╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: ebb18576-ff58-4094-8a6e-36c27f967cce                                                                     │
│  Agent: Weather presenter                                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: a34de1e7-fd36-4e2a-a932-293c7a3e221b                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): 

AuthenticationError: litellm.AuthenticationError: AuthenticationError: Litellm_proxyException - The api_key client option must be set either by passing api_key to the client or by setting the LITELLM_PROXY_API_KEY environment variable